In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [2]:
OPT_INDEX = '/Lun0/zhiyong/libri'
# data_dir = '/Lun0/zhiyong/dataset/vox2/dev/aac'
# data_dir_1 = '/Lun0/zhiyong/dataset/vox1/dev/wav'
# data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'
data_dir_100 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100'
data_dir_360 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360'
data_dir_o500 = '/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500'
new_start_id = 7911 #vox with 0-7322 in with 7323-7910


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
# val_utt_num = 200

In [18]:
spk2utt = {}
# spk2utt_train = {}
# spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

label_dir = glob.glob(data_dir_100+'/*')
label = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_360+'/*')
label_1 = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_o500+'/*')
label_2 = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = glob.glob(os.path.join(data_dir_100, i)+'/*/*.flac')

for count_1, i in enumerate(label_1):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1] = glob.glob(os.path.join(data_dir_360, i)+'/*/*.flac')

for count_2, i in enumerate(label_2):
    with open(spk2id, 'a') as f:
        f.write(str(new_start_id+count+1+count_1+1+count_2)+','+i+'\n')
    spk2utt[new_start_id+count+1+count_1+1+count_2] = glob.glob(os.path.join(data_dir_o500, i)+'/*/*.flac')

In [4]:
# should be class:
251+921+1166

2338

In [7]:
7910 + 2338

10248

In [13]:
len(spk2utt)

2338

In [14]:
spk2utt.keys()

dict_keys([7911, 7912, 7913, 7914, 7915, 7916, 7917, 7918, 7919, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7927, 7928, 7929, 7930, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7940, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7971, 7972, 7973, 7974, 7975, 7976, 7977, 7978, 7979, 7980, 7981, 7982, 7983, 7984, 7985, 7986, 7987, 7988, 7989, 7990, 7991, 7992, 7993, 7994, 7995, 7996, 7997, 7998, 7999, 8000, 8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8010, 8011, 8012, 8013, 8014, 8015, 8016, 8017, 8018, 8019, 8020, 8021, 8022, 8023, 8024, 8025, 8026, 8027, 8028, 8029, 8030, 8031, 8032, 8033, 8034, 8035, 8036, 8037, 8038, 8039, 8040, 8041, 8042, 8043, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8054, 8055, 8056, 8057, 8058, 8059, 8060, 8061, 8062, 8063, 8064, 8065, 8066, 8067, 8068, 8069, 8070, 8071, 8072, 8073, 8074, 8075,

In [19]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [22]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)
# for i in range(val_utt_num):
#     spk2utt_val[i] = [spk2utt[i][0]]
#     spk2utt_train[i] = spk2utt[i][1:]

In [23]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

281241


In [24]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [11]:
# count = 0
# count_list = []
# for i in spk2utt_val:
#     count += len(spk2utt_val[i])
#     count_list.append(len(spk2utt[i]))
# print(count)

200


In [12]:
# with open(spk2utt_val_dict, 'wb') as handle:
#     pickle.dump(spk2utt_val, handle)

In [13]:
music_dir = '/Lun0/zhiyong/dataset/musan/music'
noise_dir = '/Lun0/zhiyong/dataset/musan/noise'
speech_dir = '/Lun0/zhiyong/dataset/musan/speech'
rir_dir = '/Lun0/zhiyong/dataset/RIRS_NOISES'

music_dict_out = os.path.join(OPT_INDEX, 'music_dict')
noise_dict_out = os.path.join(OPT_INDEX, 'noise_dict')
babble_dict_out = os.path.join(OPT_INDEX, 'babble_dict')
rir_dict_out = os.path.join(OPT_INDEX, 'rir_dict')

In [14]:
music_dict = {}
noise_dict = {}
babble_dict = {}
rir_dict = {}

In [15]:
tmp_dir_list = glob.glob(music_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    music_dict[count] = this_dir
with open(music_dict_out, 'wb') as handle:
    pickle.dump(music_dict, handle)

In [16]:
tmp_dir_list = glob.glob(noise_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    noise_dict[count] = this_dir
with open(noise_dict_out, 'wb') as handle:
    pickle.dump(noise_dict, handle)

In [17]:
tmp_dir_list = glob.glob(speech_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    babble_dict[count] = this_dir
with open(babble_dict_out, 'wb') as handle:
    pickle.dump(babble_dict, handle)

In [18]:
tmp_dir_list = glob.glob(rir_dir+'/simulated_rirs/smallroom/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    rir_dict[count] = this_dir
tmp_dir_list = glob.glob(rir_dir+'/simulated_rirs/mediumroom/*/*.wav')
for count_2, this_dir in enumerate(tmp_dir_list):
    rir_dict[count +1 + count_2] = this_dir

with open(rir_dict_out, 'wb') as handle:
    pickle.dump(rir_dict, handle)

In [25]:
music_len = os.path.join(OPT_INDEX, 'music_len')
noise_len = os.path.join(OPT_INDEX, 'noise_len')
babble_len = os.path.join(OPT_INDEX, 'babble_len')
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# spk2utt_val_len = os.path.join(OPT_INDEX, 'spk2utt_val_len')

music_len_dict = {}
noise_len_dict = {}
babble_len_dict = {}
spk2utt_train_len_dict = {}
# spk2utt_val_len_dict = {}

In [20]:
with open(music_dict_out, 'rb') as handle:
    music_dict = pickle.load(handle)

for i in music_dict:
    music_len_dict[i] = librosa.core.load(music_dict[i], sr=16000)[0].shape[0]

with open(music_len, 'wb') as handle:
    pickle.dump(music_len_dict, handle)

In [21]:
with open(noise_dict_out, 'rb') as handle:
    noise_dict = pickle.load(handle)

for i in noise_dict:
    noise_len_dict[i] = librosa.core.load(noise_dict[i], sr=16000)[0].shape[0]

with open(noise_len, 'wb') as handle:
    pickle.dump(noise_len_dict, handle)

In [22]:
with open(babble_dict_out, 'rb') as handle:
    babble_dict = pickle.load(handle)

for i in babble_dict:
    babble_len_dict[i] = librosa.core.load(babble_dict[i], sr=16000)[0].shape[0]

with open(babble_len, 'wb') as handle:
    pickle.dump(babble_len_dict, handle)

In [26]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [15]:
with open(spk2utt_val_dict, 'rb') as handle:
    spk2utt_val = pickle.load(handle)

for i in spk2utt_val:
    for utt in spk2utt_val[i]:
        spk2utt_val_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_val_len, 'wb') as handle:
    pickle.dump(spk2utt_val_len_dict, handle)

In [49]:
# trial dict
trial_dir = '/Lun0/zhiyong/dataset/vox1/test/wav'
trial_dict_out = os.path.join(OPT_INDEX, 'trial_dict')
trial_dict = {}

tmp_dir_list = glob.glob(trial_dir+'/*/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    trial_dict[count] = this_dir
with open(trial_dict_out, 'wb') as handle:
    pickle.dump(trial_dict, handle)

In [52]:
len(trial_dict)

4874

# Iterable Dataset & Make train data

In [7]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
                out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(400, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
#             concat_utt = np.zeros(0)
            if get_count == 0:
                utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
                utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            try:
                utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            except Exception:
                get_count += 1
                print(utt_dir, off, dur)
                continue
#                 raise Exception(utt_dir+' '+str(off)+' '+str(dur))       
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [4]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [5]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [109]:
# config = {}

# config['sr'] = 16000
# config['repeats'] = 150
# config['batch_size'] = 128
# config['extended_prefectch'] = 1.0

# def iter_data_preload(dataset, i, start):
#     dataset.get_random_list()
#     try:
#         start_time = time.time()
#         for count, (data, label) in enumerate(dataset):
#             with open('/Lun0/zhiyong/libri/train_data/'+str(i)+str('_')+str(start+1+count), 'wb') as handle:
#                 pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
# #             print(time.time()-start_time)
#             with open('/Lun0/zhiyong/libri/tmp_data_errlog_1', 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
#             start_time = time.time()
#             if start+1+count == 2738:
#                 break
#     except Exception:
#         with open('/Lun0/zhiyong/libri/tmp_data_errlog_1', 'a') as f:
#             traceback.print_exc(file=f)
#         raise Exception

# dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# # dataset.noise_data_preload2mem()
# dataset.noise_data_preload()

# # processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(10)]
# start_list = {1:2001, 3:1087, 7:923, 9:1288}
# processes = [Process(target = iter_data_preload, args = (dataset, i, start_list[i])) for i in [1, 3, 7, 9]]
# start_time = time.time()
# [p.start() for p in processes]
# joined = [p.join() for p in processes]
# print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/4788/94904/4788-94904-0018.flac 1.40625 4.5
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.2633125 3.57
10299.95515537262


In [8]:
config = {}

config['sr'] = 16000
config['repeats'] = 150
config['batch_size'] = 1
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/libri/train_data_sparse/'+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open('/Lun0/zhiyong/libri/tmp_data_errlog_1', 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
    except Exception:
        with open('/Lun0/zhiyong/libri/tmp_data_errlog_1', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(10)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500/2288/139962/2288-139962-0008.flac 7.89075 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-other-500/7001/12337/7001-12337-0095.flac 5.3880625 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1987/144135/1987-144135-0100.flac 0.7945 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.3838125 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/1212/75242/1212-75242-0002.flac 4.8921875 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-100/4788/94904/4788-94904-0018.flac 1.4620625 4.99
/Lun0/zhiyong/SdSV_2020_deepmine/librispeech/LibriSpeech/train-clean-360/4595/45218/4595-45218-0016.flac 2.82675 4.99
/Lun0/zhiyong/SdSV_2

# Make train index

In [113]:
2739*10

27390

In [9]:
2338*1500

3507000

In [12]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_sparse')
expected_len = 3507000
workers = 10
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_sparse.csv')

In [13]:
len(glob.glob(train_data_dir+'/*'))

3507000

In [14]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [15]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [16]:
count

3507000

In [108]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
# train_data_dir = os.path.join(OPT_INDEX, 'tmp_data_2')
# expected_len = 85810
# workers = 10
# single_worker_len = int(expected_len / workers)
# output = os.path.join(OPT_INDEX, 'tmp_data_2_csv.csv')

In [109]:
# assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [110]:
# count = 0
# with open(output, 'w') as f:
#     for i in range(workers):
#         for j in range(single_worker_len):
#             path = os.path.join(train_data_dir, str(i)+'_'+str(j))
#             assert os.path.isfile(path)
#             f.write(path+'\n')
#             count += 1

In [111]:
# count

85810

# Test

In [ ]:
# from torch.utils.data import Dataset, DataLoader

In [ ]:
# class PickleDataSet(Dataset):
#     def __init__(self, spkid2utt_csv):
#         self.spkid2utt = pd.read_csv(spkid2utt_csv, header=None)
        
#     def __getitem__(self, index):
#         path = self.spkid2utt[0][index]
#         with open(path, 'rb') as handle:
#             batch_data = pickle.load(handle)
#         batch_feats = batch_data[0].astype(np.float32)
#         batch_labels = batch_data[1].astype(np.int64)
       
#         return batch_feats, batch_labels

#     def __len__(self):
#         return len(self.spkid2utt)

In [ ]:
# pickle_dataset = PickleDataSet(output)

In [ ]:
# length = len(pickle_dataset)

In [ ]:
# length_list = []

In [ ]:
# for i in range(length):
#     length_list.append(pickle_dataset[i][0].shape[1])
#     if (i+1) % 1000 == 0:
#         print(i+1)

In [ ]:
# len(length_list)

In [ ]:
# for i in length_list:
#     print(i)

# Make validation data (Need only once)

In [20]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
#                 out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
#             concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [21]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [22]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

In [23]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/vox_small_novad_sparse/val_data/'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
            with open('/Lun0/zhiyong/vox_small_novad_sparse/tmp_data_errlog', 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
    except Exception:
        with open('/Lun0/zhiyong/vox_small_novad_sparse/tmp_data_errlog', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
8.780897378921509


# Make validation index (Need only once)

In [24]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'val_data')
expected_len = 1000
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_csv.csv')

In [25]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [26]:
count = 0
with open(output, 'w') as f:
    for j in range(single_worker_len):
        path = os.path.join(train_data_dir, str(j))
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1

In [27]:
count

1000

# Make trial data (Need only once)

In [82]:
fft = librosa.get_fftlib()
class VoxIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
                out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = in_wav

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [77]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX, 'rir_dict')

In [78]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX, 'babble_len')

In [87]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None

trial_dict_dir = '/Lun0/zhiyong/dataset/trial_dict'

def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    for i in trial_dict:
        data = dataset.process_one_utt(trial_dict[i])
        lbpart = trial_dict[i].split('/')[-3:]
        lbpart = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
        label = lbpart[:-4]
        with open('/Lun0/zhiyong/dataset/trial_data/'+str(i), 'wb') as handle:
            pickle.dump((data.astype(np.float16), [label]), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

0.07287907600402832
0.022907257080078125
0.025229215621948242
0.03087759017944336
0.027460336685180664
0.012605428695678711
0.013921260833740234
0.018674850463867188
0.025691986083984375
0.01715874671936035
0.022578716278076172
0.03155040740966797
0.013101339340209961
0.021047592163085938
0.020425796508789062
0.014108896255493164
0.018069982528686523
0.028415679931640625
0.0164029598236084
0.023502588272094727
0.024616718292236328
0.013360738754272461
0.042522430419921875
0.023035049438476562
0.040648698806762695
0.01689767837524414
0.028036832809448242
0.02653956413269043
0.08423042297363281
0.024197816848754883
0.0241091251373291
0.04253411293029785
0.03831362724304199
0.014290809631347656
0.01449894905090332
0.06771516799926758
0.014318466186523438
0.022284984588623047
0.019834041595458984
0.02354121208190918
0.012340068817138672
0.01557016372680664
0.023464202880859375
0.01560068130493164
0.017389774322509766
0.014495849609375
0.02409219741821289
0.014666557312011719
0.014875173568

0.015488147735595703
0.009747028350830078
0.013583660125732422
0.01604628562927246
0.010938882827758789
0.016330718994140625
0.018108606338500977
0.024467945098876953
0.015343666076660156
0.020621299743652344
0.012320280075073242
0.009239673614501953
0.010608434677124023
0.012408018112182617
0.009007692337036133
0.009940624237060547
0.023001909255981445
0.009037256240844727
0.013557195663452148
0.012632608413696289
0.011828184127807617
0.01585078239440918
0.00923299789428711
0.009802579879760742
0.018346071243286133
0.010751485824584961
0.014223575592041016
0.023897171020507812
0.01324915885925293
0.013222217559814453
0.017880678176879883
0.012619972229003906
0.01996016502380371
0.012471199035644531
0.01659870147705078
0.01203155517578125
0.026020288467407227
0.009439945220947266
0.018280982971191406
0.013326883316040039
0.014516353607177734
0.016368865966796875
0.009016990661621094
0.014873981475830078
0.013907432556152344
0.03354215621948242
0.009061813354492188
0.014988183975219727


0.03825807571411133
0.017183542251586914
0.01438283920288086
0.024358034133911133
0.03087592124938965
0.017307043075561523
0.012839555740356445
0.016971826553344727
0.028851985931396484
0.012452840805053711
0.016255855560302734
0.01796245574951172
0.020953655242919922
0.032532453536987305
0.00838613510131836
0.009426355361938477
0.007505655288696289
0.0780487060546875
0.0366055965423584
0.033304691314697266
0.11432552337646484
0.02301621437072754
0.02089548110961914
0.030603647232055664
0.05159616470336914
0.014839649200439453
0.01777815818786621
0.06510543823242188
0.02220439910888672
0.021721839904785156
0.03770709037780762
0.03400111198425293
0.013321876525878906
0.013467550277709961
0.02005600929260254
0.015294551849365234
0.040691375732421875
0.023011207580566406
0.020557880401611328
0.022239208221435547
0.022853612899780273
0.012698888778686523
0.022971630096435547
0.04580879211425781
0.02337932586669922
0.0379328727722168
0.01364588737487793
0.029843568801879883
0.01499652862548

0.06371521949768066
0.05131840705871582
0.03843569755554199
0.014810323715209961
0.02761554718017578
0.012833118438720703
0.012442350387573242
0.02368640899658203
0.021667957305908203
0.03663229942321777
0.015869855880737305
0.016068696975708008
0.01753401756286621
0.023042678833007812
0.014765739440917969
0.028501510620117188
0.025823116302490234
0.014516830444335938
0.02416706085205078
0.020163774490356445
0.012816667556762695
0.01602792739868164
0.021846532821655273
0.013501882553100586
0.027643918991088867
0.023738861083984375
0.018512487411499023
0.02137899398803711
0.015130043029785156
0.021465063095092773
0.02170109748840332
0.04655790328979492
0.014817953109741211
0.01321268081665039
0.01347041130065918
0.012141942977905273
0.008195638656616211
0.025912046432495117
0.08602046966552734
0.029409170150756836
0.03990650177001953
0.028842687606811523
0.024647951126098633
0.10697770118713379
0.019201278686523438
0.08666682243347168
0.04427647590637207
0.026543855667114258
0.028393507

0.08216714859008789
0.05129599571228027
0.050977230072021484
0.033893585205078125
0.09801626205444336
0.02569413185119629
0.12413573265075684
0.04647660255432129
0.09634804725646973
0.032402992248535156
0.02441120147705078
0.020857572555541992
0.03323054313659668
0.04268765449523926
0.057228803634643555
0.027840137481689453
0.06336164474487305
0.02483677864074707
0.018707752227783203
0.0383305549621582
0.028452157974243164
0.024735689163208008
0.04900646209716797
0.04793548583984375
0.04253745079040527
0.02677774429321289
0.035401105880737305
0.024541378021240234
0.025503873825073242
0.036878347396850586
0.022688865661621094
0.02305459976196289
0.027027606964111328
0.01900172233581543
0.038512229919433594
0.02403736114501953
0.02490711212158203
0.023688316345214844
0.038759469985961914
0.026224374771118164
0.02048516273498535
0.015273094177246094
0.01764702796936035
0.020224809646606445
0.01542973518371582
0.017786264419555664
0.018315792083740234
0.013355255126953125
0.017240524291992

...

0.014484643936157227
0.008175134658813477
0.016806364059448242
0.014342784881591797
0.00942683219909668
0.009820222854614258
0.012914896011352539
0.011742830276489258
0.016219377517700195
0.01873779296875
0.019175052642822266
0.029824495315551758
0.017229080200195312
0.021924495697021484
0.02479696273803711
0.015429496765136719
0.030822038650512695
0.012513160705566406
0.032344818115234375
0.023973941802978516
0.012398481369018555
0.04993486404418945
0.014546632766723633
0.013065814971923828
0.02096867561340332
0.020741701126098633
0.026541948318481445
0.012352466583251953
0.024150371551513672
0.03346729278564453
0.02159571647644043
0.0556337833404541
0.029448509216308594
0.012563705444335938
0.016297340393066406
0.012530803680419922
0.021334409713745117
0.022617578506469727
0.014428377151489258
0.01294851303100586
0.015323162078857422
0.023638010025024414
0.014261960983276367
0.015328168869018555
0.024859905242919922
0.02049708366394043
0.02735447883605957
0.021831750869750977
0.01946

0.01779341697692871
0.04741406440734863
0.022950410842895508
0.021553754806518555
0.014911413192749023
0.022769927978515625
0.05577445030212402
0.04092097282409668
0.035112619400024414
0.01647353172302246
0.02602696418762207
0.01828789710998535
0.03457140922546387
0.016360998153686523
0.01477503776550293
0.025612592697143555
0.03476452827453613
0.017386913299560547
0.04560112953186035
0.028133392333984375
0.03404951095581055
0.03560972213745117
0.02205801010131836
0.05791926383972168
0.020664453506469727
0.015633106231689453
0.0388031005859375
0.02399468421936035
0.032187700271606445
0.013055562973022461
0.043715715408325195
0.01747441291809082
0.012763023376464844
0.013087272644042969
0.032958269119262695
0.028151273727416992
0.03212571144104004
0.0203397274017334
0.05665135383605957
0.0297696590423584
0.07439398765563965
0.01578378677368164
0.0375370979309082
0.012744903564453125
0.04756641387939453
0.07330799102783203
0.1005716323852539
0.06286287307739258
0.037529945373535156
0.062

0.013020753860473633
0.012336492538452148
0.0075571537017822266
0.01329183578491211
0.007775783538818359
0.012459754943847656
0.01592230796813965
0.012757301330566406
0.007819652557373047
0.00877833366394043
0.00915217399597168
0.013603448867797852
0.008455991744995117
0.020301103591918945
0.020557165145874023
0.012262821197509766
0.009631872177124023
0.008740425109863281
0.011198997497558594
0.015882253646850586
0.00813746452331543
0.017527103424072266
0.015496969223022461
0.015604734420776367
0.007626056671142578
0.010302066802978516
0.012313127517700195
0.009665250778198242
0.024631977081298828
0.019566059112548828
0.007767200469970703
0.0077114105224609375
0.02915191650390625
0.01859760284423828
0.035521507263183594
0.017155885696411133
0.013559341430664062
0.013555765151977539
0.013937950134277344
0.02116560935974121
0.012603282928466797
0.013136148452758789
0.02286076545715332
0.013257265090942383
0.016905784606933594
0.04301619529724121
0.0175018310546875
0.01627206802368164
0.0

0.008137226104736328
0.008273601531982422
0.008854866027832031
0.011574745178222656
0.009765386581420898
0.008453369140625
0.025687217712402344
0.011217832565307617
0.009511947631835938
0.010664224624633789
0.021167993545532227
0.008888006210327148
0.01261591911315918
0.00804281234741211
0.01900506019592285
0.012243509292602539
0.021486759185791016
0.012070417404174805
0.010683536529541016
0.021074295043945312
0.008438587188720703
0.012056112289428711
0.010554790496826172
0.022570371627807617
0.008146047592163086
0.01270294189453125
0.018619537353515625
0.008700847625732422
0.010376453399658203
0.023976802825927734
0.03686094284057617
0.03320956230163574
0.018467187881469727
0.019283771514892578
0.018865108489990234
0.02293992042541504
0.02224278450012207
0.02098369598388672
0.023928165435791016
0.015554666519165039
0.01915884017944336
0.02748847007751465
0.029588937759399414
0.016643285751342773
0.01741623878479004
0.04029035568237305
0.046560049057006836
0.10404658317565918
0.0271921

0.019851207733154297
0.027615070343017578
0.016291141510009766
0.02029252052307129
0.015343904495239258
0.020166635513305664
0.014487266540527344
0.02420639991760254
0.013915538787841797
0.02392435073852539
0.013460159301757812
0.02347564697265625
0.012786149978637695
0.012405872344970703
0.03556060791015625
0.012409210205078125
0.01668572425842285
0.02846240997314453
0.02585911750793457
0.013944149017333984
0.030778884887695312
0.04447293281555176
0.03174233436584473
0.05433344841003418
0.019062280654907227
0.016187429428100586
0.08639788627624512
0.028354644775390625
0.01448965072631836
0.03028583526611328
0.0243682861328125
0.020796537399291992
0.02422499656677246
0.053694963455200195
0.01396799087524414
0.015400409698486328
0.012473583221435547
0.013853073120117188
0.014121055603027344
0.023286104202270508
0.014570474624633789
0.024480342864990234
0.028972387313842773
0.015004634857177734
0.014763593673706055
0.025196313858032227
0.02899312973022461
0.04214811325073242
0.0135796070

# Make trial index (Need only once)

In [88]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'trial_data')
expected_len = 4874
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_data_csv.csv')

In [67]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [91]:
count = 0
with open(output, 'w') as f:
    for j in range(single_worker_len):
        path = os.path.join(train_data_dir, str(j))
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1

In [92]:
count

4874

# Make PLDA

In [46]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
                out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
            concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [47]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX, 'rir_dict')

In [48]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX, 'babble_len')

In [49]:
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')

with open(spk2utt_dict, 'rb') as handle:
    spk2utt_dict_data = pickle.load(handle)
count = 0
for i in spk2utt_dict_data:
    count += len(spk2utt_dict_data[i])

In [50]:
count

1276888

In [53]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None

# trial_dict_dir = '/Lun0/zhiyong/dataset/trial_dict'

def trial_data_preload(dataset, i, spk2utt_dict_data):
        
    for i in spk2utt_dict_data:
        for j, line in enumerate(spk2utt_dict_data[i]):
#             data = dataset.process_one_utt(line)
            data = dataset.process_one_utt_noVAD(line)
            lbpart = line.split('/')[-3:]
            lbpart = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
            label = lbpart[:-4]
            with open('/Lun0/zhiyong/dataset/plda_full_data_noVAD/'+str(i)+'_'+str(j), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
        if ((i+1) % 100) == 0:    
            print(i+1)

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, spk2utt_dict_data)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
22095.06617307663


# Make PLDA index

In [54]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'plda_full_data_noVAD')
expected_len = 1276888
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'plda_full_data_noVAD.csv')

In [55]:
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')

with open(spk2utt_dict, 'rb') as handle:
    spk2utt_dict_data = pickle.load(handle)

In [56]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [57]:
count = 0
with open(output, 'w') as f:
    for i in spk2utt_dict_data:
        for j in range(len(spk2utt_dict_data[i])):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [58]:
count

1276888